# Microservices + Events + Docker
https://www.youtube.com/watch?v=sSm2dRarhPo

## Monolith vs Microservices
Microservices: functional decomposition

### Benefits
Trio: 
- Architecture
- Process: Agile, Continuous delivery, ...
- Organization: Small, autonomous, teams (6~8)


### Drawbacks
- __Complexity__ of developing a distributed system
    - Implementing inter-process communication
    - Handling partial failures
- Complexity of implementing business transactions that _span multiple databases_ (without 2PC)
- Complexity of testing a distributed system
- Complexity of deploying and operating a distributed system
- Managing the development and deployment of features that span multiple services

### Issue to address
- How to deploy the services?
- How do the services communicate? 
- How do clients of the application communicate with the services? 
- How to partition the system into services? 
- How to deal with distributed data management problems

## Event-driven Microservices
### Data management patterns
__Single shared database__ or __Database per service__

- Shared Database
![Shared Database](Asset/Microservices - Shared database.png)


- Database per service
    - each services should have its own private data that is only accessible through that service's API
    - Loose coupling but more complex
    - 2PC (aka. distributed transactions) is not viable choice for most modern applications - 2 Phase Commit / the cap the cap theorem / data consistency

### Event-driven architecture
- Whenever something significant happens (state changes), publish an event
- another service can consume this event and react accordingly, update its own data

- Use event-driven, eventually consistent order processing
    - Ebay model many years ago
![Event Driven](Asset/Microservices - Event Driven.png)

- __Problem__: How to atomically update database and publish an event?
    - Dual write problem
    - Tradition way: distributed transaction [Begin - update DB - update message broker - commit transaction]
    - but no distributed transaction in modern application
    
- Reliably publish events when state changes

### Use event-sourcing
Event centric approach to persistence. i.e., order = sequence changing of an event 
- Event table:

| Aggregate id | Aggregate type | Event id | Event type | Event Data |
|-----|-------|-----|---------------|-----|
| 101 | Order | 901 | OrderCreated  | ... |
| 101 | Order | 902 | OrderApproved | ... |
| 101 | Order | 903 | OrderShipped  | ... |

- Subscriber: Pull the event table

- Reply events to recreate state
    - Read Events to recompute the current state
    - Periodically snapshot to avoid loading all events

### Benefits of event sourcing
- Solves data consistency issues in a Microservices/NoSQL based architecture
- Reliable event publishing: publishes events needed by predictive analytics etc, user notifications, ...
- Eliminates O/R mapping problem (mostly)
- Reifies state changes:
    - Build in, reliable audit log
    - temporal queries
- Preserved history => More easily implement future requirements

### Drawbacks of event sourcing
- Requires application rewrite
- Weird and unfamiliar style of programming
- Events = a historical record of your bad design decisions
- Must handle duplicate events: idempotent handlers or duplicate detection
- Querying the event store can be challenging


### Queries?
- Find recent, valuable customers:
```SQL
SELECT * FROM CUSTOMERc, ORDER o 
WHERE c.id = o.ID AND o.ORDER_TOTAL > 100000 AND o.STATE = 'SHIPPED' AND c.CREATION_DATE > ?
```
- Microservices: No longer easy 
    - in 2 databases
    - Event Source: Writing a query involving the current state is non-trival

### Command Query Responsibility Segregation (CQRS)
![CQRS](Asset/Microservices - CQRS.png)


## Developing and deploying microservices using Docker
TBD

# Dockerfile

- Dockerfile -> Image -> Container


- Shell form: `<instruction> <command>`
    - When instruction is executed in shell form it calls `/bin/sh -c <command>` under the hood and normal shell processing happens.

```Dockerfile
RUN apt-get install python3
CMD echo "Hello world"
ENTRYPOINT echo "Hello world"

...

ENV name Stevo
ENTRYPOINT echo "Hello, $name"
# Hello, Stevo   # Note that variable name is replaced with its value.
```

- Exec form: `<instruction> ["executable", "param1", "param2", ...]`
    - This is the preferred form for CMD and ENTRYPOINT instructions.
    
```Dockerfile
RUN ["apt-get", "install", "python3"]
CMD ["/bin/echo", "Hello world"]
ENTRYPOINT ["/bin/echo", "Hello world"]

...

ENV name Stevo
ENTRYPOINT ["/bin/echo", "Hello, $name"]
# Hello, $name   # Note that variable name is not substituted.
```


## Commands

Common commands are: `RUN`, `FROM`, `MAINTAINER`, `ADD`, `CMD`, `ENTRYPOINT`, `ENV`, `EXPOSE`, `USER`, `VOLUME`, `WORKDIR`, 
...


### `FROM` 
It defines the base image tuo use to start the build process
```Dockerfile
# Usage: FROM [image name]
FROM ubuntu
```

### `RUN`
Executing directive for Dockerfiles. It takes a command as its argument and runs it from the image. Unlike `CMD`, it actually __is__ used to build the image

RUN executes command(s) in a new layer and creates a new image. E.g., it is often used for installing software packages.

```Dockerfile
# RUN <command> (shell form)
# RUN ["executable", "param1", "param2"] (exec form)
RUN apt-get update
```

### `CMD`
Similar to `RUN`, `CMD` can be used for executing a specific command. However, unlike `RUN`, it isn't executed druing build, but when a container is instantiated using the image being built.

CMD sets default command and/or parameters, which can be overwritten from command line when docker container runs.

- CMD instruction allows you to set a default command, which will be executed only when you run container without specifying a command.
- Docker container runs with a command, the default command will be ignored. 
- If Dockerfile has more than one CMD instruction, all but last CMD instructions are ignored.


```Dockerfile
# CMD ["executable","param1","param2"] (exec form, preferred)
# CMD ["param1","param2"] (sets additional default parameters for ENTRYPOINT in exec form)
# CMD command param1 param2 (shell form)

RUN echo "Hello World"
```

### `ENTRYPOINT`
It sets the concrete default application that is used everytime a container is created using the image

ENTRYPOINT configures a container that will run as an executable.

- ENTRYPOINT command and parameters are __not__ ignored when Docker container runs with command line parameters. 

```Dockerfile
# ENTRYPOINT ["executable", "param1", "param2"] (exec form, preferred)
# ENTRYPOINT command param1 param2 (shell form)

# Usage: ENTRYPOINT application "argument", "argument", ...
# Arguments are OPTIONAL. They can be provided by CMD or during the creation of a container
ENTRYPOINT echo

# USAGE example with CMD:
# Arguments set with CMD can be overridden during CMD "Hello World"
CMD "Hello World"
ENTRYPOINT echo
```

- __Exec form__ of ENTRYPOINT allows you to set commands and parameters and then use either form of CMD to set additional parameters that are more likely to be changed. 
    - ENTRYPOINT arguments are always used, while CMD ones can be __overwritten by command line arguments__ provided when Docker container runs.
    
    
- __Shell form__ of ENTRYPOINT ignores any CMD or docker run command line arguments.

```Dockerfile
ENTRYPOINT ["/bin/echo", "Hello"]
CMD ["world"]

# docker run -it <image>  >>> Hello world
# docker run -it <image> Steve  >>> Hello Steve
```

### `COPY`
COPY takes in a src and destination. It only lets you copy in a local file or directory from your host (the machine building the Docker image) into the Docker image itself.
- More explicit

### `ADD`
- Copy from local fs
- Copy from URL
- Extract a tar: `ADD rootfs.tar.gz /`


### `ENV`
`ENV key value`

### `WORKDIR`
Sets the working directory for any RUN, CMD, ENTRYPOINT, COPY and ADD instructions that follow it in the Dockerfile

- The WORKDIR instruction can resolve environment variables previously set using ENV. You can only use environment variables explicitly set in the Dockerfile
 
```Dockerfile
WORKDIR /a
WORKDIR b
WORKDIR c
RUN pwd
# /a/b/c
```

### `EXPOSE`
associate a specified port to enable networking between the running process inside the container and the host

`EXPOSE 5000`

### `MAINTAINER`
`MAINTAINER author_name`

### `USER`
`USER [UID]`: sets the user name (or UID) and optionally the user group (or GID) to use when running the image and for any RUN, CMD and ENTRYPOINT instructions that follow it in the Dockerfile.


### `VOLUME`
creates a mount point with the specified name and marks it as holding externally mounted volumes from native host or other containers. 


## RUN vs CMD vs ENTRYPOING
Use RUN instructions to build your image by adding layers on top of initial image.

Prefer ENTRYPOINT to CMD when building executable Docker image and you need a command always to be executed. Additionally use CMD if you need to provide extra default arguments that could be overwritten from command line when docker container runs.

Choose CMD if you need to provide a default command and/or arguments that can be overwritten from command line when docker container runs.

## Use multi-stage builds
https://docs.docker.com/develop/develop-images/multistage-build/

With multi-stage builds, you use multiple FROM statements in your Dockerfile. Each FROM instruction can use a different base, and each of them begins a new stage of the build. You can selectively copy artifacts from one stage to another, leaving behind everything you don’t want in the final image.

```Dockerfile
FROM golang:1.7.3
WORKDIR /go/src/github.com/alexellis/href-counter/
RUN go get -d -v golang.org/x/net/html  
COPY app.go .
RUN CGO_ENABLED=0 GOOS=linux go build -a -installsuffix cgo -o app .

FROM alpine:latest  
RUN apk --no-cache add ca-certificates
WORKDIR /root/
COPY --from=0 /go/src/github.com/alexellis/href-counter/app .
CMD ["./app"]  
```
The second FROM instruction starts a new build stage with the alpine:latest image as its base. The COPY --from=0 line copies just the built artifact from the previous stage into this new stage. The Go SDK and any intermediate artifacts are left behind, and not saved in the final image.

### Name your build stages
```Dockerfile
FROM python:3.6-alpine as base

FROM base as builder
RUN mkdir /install
WORKDIR /install
COPY requirements.txt /requirements.txt
RUN pip install --install-option="--prefix=/install" -r /requirements.txt

FROM base
COPY --from=builder /install /usr/local
COPY src /app
WORKDIR /app
CMD ["gunicorn", "-w 4", "main:app"]
```

### Stop at a specific build stage
`$ docker build --target builder -t alexellis2/href-counter:latest .`


### Use an external image as a “stage”
`COPY --from=nginx:latest /etc/nginx/nginx.conf /nginx.conf`